In [3]:
pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 852.9 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import torchtext
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from torchtext.vocab import Vectors
from tqdm import tqdm_notebook
import pandas as pd

In [5]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes, batch_size):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes, bias = True)
        self.array_like = np.zeros((batch_size, input_size))

    def forward(self, x):
        output = self.linear(x)
        activated = torch.nn.functional.sigmoid(output)
        return activated

    def predict(self, x):
        output = self.forward(x)
        logits = torch.nn.functional.log_softmax(output, dim = 1)
        return logits.max(1)[1] + 1

    def binarize_occurrences(self, indices):
        occurrences = self.array_like.copy()
        for idx, entry in enumerate(indices): occurrences[idx][entry] = 1
        return occurrences

    def batch_to_input(self, batch, train = True):
        word_indices = batch.text.data.numpy().T
        x = self.binarize_occurrences(word_indices)
        if train:
            return Variable(torch.FloatTensor(x)), batch.label
        else:
            return Variable(torch.FloatTensor(x))

    def train(self, train_iter, val_iter, num_epochs, learning_rate = 1e-3):
        criterion = torch.nn.NLLLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
        loss_vec = []

        for epoch in range(1, num_epochs+1):
            epoch_loss = 0
            for batch in train_iter:
                x, y = self.batch_to_input(batch, train = True)

                optimizer.zero_grad()

                output = self.forward(x)

                loss = criterion(output, y-1)
                loss.backward()

                optimizer.step()
                epoch_loss += loss.data[0]
            self.model = model

            loss_vec.append(epoch_loss / len(train_iter))
            if epoch % 1 == 0:
                acc = self.validate(val_iter)
                print('Epoch {} loss: {} | Valid acc: {}'.format(epoch, loss_vec[epoch-1], acc))

        plt.plot(range(len(loss_vec)), loss_vec)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()
        print('\nModel trained.\n')
        self.loss_vec = loss_vec

    def test(self, test_iter):
        "All models should be able to be run with following command."
        upload, trues = [], []
        # Update: for kaggle the bucket iterator needs to have batch_size 10
        for batch in test_iter:
            # Your prediction data here (don't cheat!)
            x, y = self.batch_to_input(batch, train = False), batch.label
            preds = self.predict(x)
            upload += list(preds.data.numpy())
            trues += list(y.data.numpy())

        correct = sum([1 if i == j else 0 for i, j in zip(upload, trues)])
        accuracy = correct / len(trues)
        print('Test Accuracy:', accuracy)

        with open("predictions.txt", "w") as f:
            for u in upload:
                f.write(str(u) + "\n")

    def validate(self, val_iter):
        y_p, y_t, correct = [], [], 0
        for batch in val_iter:
            x, y = self.batch_to_input(batch, train = False), batch.label
            probs = self.model.predict(x)[:len(y.data.numpy())]
            y_p += list(probs.data.numpy())
            y_t += list(y.data.numpy())
        correct = sum([1 if i == j else 0 for i, j in zip(y_p, y_t)])
        accuracy = correct / len(y_p)
        return accuracy

In [6]:
train_df = pd.read_csv("/train.csv")
validation_df = pd.read_csv("/validation.csv")

train_df.drop(columns = "idx")
validation_df.drop(columns = "idx")


test_df = pd.read_csv("/IMDB Dataset.csv")
test_df.rename(columns={'review': 'sentence', 'sentiment': 'label'}, inplace=True)
test_df['label'] = test_df['label'].map({'negative': 0, 'positive': 1})
output_path = "/content/modified_imdb_dataset.csv"
test_df.to_csv(output_path, index=False)

In [7]:
train_df["sentence"] = train_df["sentence"].str.lower()
validation_df["sentence"] = validation_df["sentence"].str.lower()
test_df["sentence"] = test_df["sentence"].str.lower()

In [8]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

train_df["text_wo_punct"] = train_df["sentence"].apply(lambda text: remove_punctuation(text))
validation_df["text_wo_punct"] = validation_df["sentence"].apply(lambda text: remove_punctuation(text))
test_df["text_wo_punct"] = test_df["sentence"].apply(lambda text: remove_punctuation(text))

In [9]:
import nltk

# Ensure you have downloaded the stopwords dataset
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
from nltk.corpus import stopwords

", ".join(stopwords.words('english'))


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [11]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

train_df["text_wo_stop"] = train_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
validation_df["text_wo_stop"] = validation_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
test_df["text_wo_stop"] = test_df["text_wo_punct"].apply(lambda text: remove_stopwords(text))

In [12]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

train_df["text_stemmed"] = train_df["text_wo_stop"].apply(lambda text: stem_words(text))
validation_df["text_stemmed"] = validation_df["text_wo_stop"].apply(lambda text: stem_words(text))
test_df["text_stemmed"] = test_df["text_wo_stop"].apply(lambda text: stem_words(text))

In [13]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize sentences
nltk.download('punkt')
train_df['tokens'] = train_df['text_stemmed'].apply(lambda x: nltk.word_tokenize(x))

# Create a word index (manual embedding)
word_set = set()
for tokens in train_df['tokens']:
    word_set.update(tokens)

word_index = {word: i+1 for i, word in enumerate(word_set)}  # Start indexing from 1

# Convert tokens to integers
train_df['indexed_tokens'] = train_df['tokens'].apply(lambda x: [word_index[token] for token in x])

# Optional: Pad sequences to ensure uniform length
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 100  # Define maximum length of sequences
train_df['padded_tokens'] = pad_sequences(train_df['indexed_tokens'], maxlen=max_length, padding='post').tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
train_df

,idx,sentence,label,text_wo_punct,text_wo_stop,text_stemmed,tokens,indexed_tokens,padded_tokens
0,0,hide new secretions from the parental units,0,hide new secretions from the parental units,hide new secretions parental units,hide new secret parent unit,"[hide, new, secret, parent, unit]","[5182, 4853, 8079, 6092, 5659]","[5182, 4853, 8079, 6092, 5659, 0, 0, 0, 0, 0, ..."
1,1,"contains no wit , only labored gags",0,contains no wit only labored gags,contains wit labored gags,contain wit labor gag,"[contain, wit, labor, gag]","[3840, 5175, 9417, 10357]","[3840, 5175, 9417, 10357, 0, 0, 0, 0, 0, 0, 0,..."
2,2,that loves its characters and communicates som...,1,that loves its characters and communicates som...,loves characters communicates something rather...,love charact commun someth rather beauti human...,"[love, charact, commun, someth, rather, beauti...","[10277, 1764, 8172, 10006, 4925, 525, 940, 6764]","[10277, 1764, 8172, 10006, 4925, 525, 940, 676..."
3,3,remains utterly satisfied to remain the same t...,0,remains utterly satisfied to remain the same t...,remains utterly satisfied remain throughout,remain utterli satisfi remain throughout,"[remain, utterli, satisfi, remain, throughout]","[10384, 8854, 5422, 10384, 10033]","[10384, 8854, 5422, 10384, 10033, 0, 0, 0, 0, ..."
4,4,on the worst revenge-of-the-nerds clichés the ...,0,on the worst revengeofthenerds clichés the fil...,worst revengeofthenerds clichés filmmakers cou...,worst revengeofthenerd cliché filmmak could dredg,"[worst, revengeofthenerd, cliché, filmmak, cou...","[9602, 1458, 8841, 256, 7831, 10747]","[9602, 1458, 8841, 256, 7831, 10747, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
67344,67344,a delightful comedy,1,a delightful comedy,delightful comedy,delight comedi,"[delight, comedi]","[6834, 8164]","[6834, 8164, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
67345,67345,"anguish , anger and frustration",0,anguish anger and frustration,anguish anger frustration,anguish anger frustrat,"[anguish, anger, frustrat]","[1091, 2584, 4029]","[1091, 2584, 4029, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
67346,67346,"at achieving the modest , crowd-pleasing goals...",1,at achieving the modest crowdpleasing goals i...,achieving modest crowdpleasing goals sets,achiev modest crowdpleas goal set,"[achiev, modest, crowdpleas, goal, set]","[2651, 8638, 6009, 187, 100]","[2651, 8638, 6009, 187, 100, 0, 0, 0, 0, 0, 0,..."
67347,67347,a patient viewer,1,a patient viewer,patient viewer,patient viewer,"[patient, viewer]","[3849, 10015]","[3849, 10015, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [15]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize sentences
nltk.download('punkt')
validation_df['tokens'] = validation_df['text_stemmed'].apply(lambda x: nltk.word_tokenize(x))

# Create a word index (manual embedding)
word_set = set()
for tokens in validation_df['tokens']:
    word_set.update(tokens)

word_index = {word: i+1 for i, word in enumerate(word_set)}  # Start indexing from 1

# Convert tokens to integers
validation_df['indexed_tokens'] = validation_df['tokens'].apply(lambda x: [word_index[token] for token in x])

# Optional: Pad sequences to ensure uniform length
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 100  # Define maximum length of sequences
validation_df['padded_tokens'] = pad_sequences(validation_df['indexed_tokens'], maxlen=max_length, padding='post').tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
validation_df

,idx,sentence,label,text_wo_punct,text_wo_stop,text_stemmed,tokens,indexed_tokens,padded_tokens
0,0,it 's a charming and often affecting journey .,1,it s a charming and often affecting journey,charming often affecting journey,charm often affect journey,"[charm, often, affect, journey]","[2560, 3067, 103, 1831]","[2560, 3067, 103, 1831, 0, 0, 0, 0, 0, 0, 0, 0..."
1,1,unflinchingly bleak and desperate,0,unflinchingly bleak and desperate,unflinchingly bleak desperate,unflinchingli bleak desper,"[unflinchingli, bleak, desper]","[419, 922, 20]","[419, 922, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2,2,allows us to hope that nolan is poised to emba...,1,allows us to hope that nolan is poised to emba...,allows us hope nolan poised embark major caree...,allow us hope nolan pois embark major career c...,"[allow, us, hope, nolan, pois, embark, major, ...","[1921, 1118, 2139, 203, 494, 1850, 1185, 2860,...","[1921, 1118, 2139, 203, 494, 1850, 1185, 2860,..."
3,3,"the acting , costumes , music , cinematography...",1,the acting costumes music cinematography an...,acting costumes music cinematography sound ast...,act costum music cinematographi sound astound ...,"[act, costum, music, cinematographi, sound, as...","[3411, 1407, 1473, 2596, 2231, 3116, 85, 3117,...","[3411, 1407, 1473, 2596, 2231, 3116, 85, 3117,..."
4,4,"it 's slow -- very , very slow .",0,it s slow very very slow,slow slow,slow slow,"[slow, slow]","[74, 74]","[74, 74, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...,...,...,...,...,...,...
867,867,has all the depth of a wading pool .,0,has all the depth of a wading pool,depth wading pool,depth wade pool,"[depth, wade, pool]","[2606, 1677, 2057]","[2606, 1677, 2057, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
868,868,a movie with a real anarchic flair .,1,a movie with a real anarchic flair,movie real anarchic flair,movi real anarch flair,"[movi, real, anarch, flair]","[3126, 745, 2274, 3048]","[3126, 745, 2274, 3048, 0, 0, 0, 0, 0, 0, 0, 0..."
869,869,a subject like this should inspire reaction in...,0,a subject like this should inspire reaction in...,subject like inspire reaction audience pianist,subject like inspir reaction audienc pianist,"[subject, like, inspir, reaction, audienc, pia...","[1362, 1425, 3142, 2840, 2517, 2184]","[1362, 1425, 3142, 2840, 2517, 2184, 0, 0, 0, ..."
870,870,... is an arthritic attempt at directing by ca...,0,is an arthritic attempt at directing by calli...,arthritic attempt directing callie khouri,arthrit attempt direct calli khouri,"[arthrit, attempt, direct, calli, khouri]","[2882, 1986, 2935, 2586, 1692]","[2882, 1986, 2935, 2586, 1692, 0, 0, 0, 0, 0, ..."


In [17]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize


# Tokenize sentences
nltk.download('punkt')

test_df['tokens'] = test_df['text_stemmed'].apply(lambda x: nltk.word_tokenize(x))

# Create a word index (manual embedding)
word_set = set()
for tokens in test_df['tokens']:
    word_set.update(tokens)

word_index = {word: i+1 for i, word in enumerate(word_set)}  # Start indexing from 1

# Convert tokens to integers
test_df['indexed_tokens'] = test_df['tokens'].apply(lambda x: [word_index[token] for token in x])

# Optional: Pad sequences to ensure uniform length
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 100  # Define maximum length of sequences
test_df['padded_tokens'] = pad_sequences(test_df['indexed_tokens'], maxlen=max_length, padding='post').tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

X_train = train_df['text_stemmed']
y_train = train_df['label']
X_valid = validation_df['text_stemmed']
y_valid = validation_df['label']


# Create the pipeline
pipeline = make_pipeline(
    TfidfVectorizer(),
    LogisticRegression(max_iter=1000)
)

# Define hyperparameters for tuning
parameters = {
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logisticregression__C': [0.1, 1.0, 10.0, 100.0],
    'logisticregression__solver': ['liblinear', 'saga'],
    'logisticregression__penalty': ['l2'],
    'logisticregression__class_weight': [None, 'balanced'],
}

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get best hyperparameters
best_params = grid_search.best_params_

# Train model with best hyperparameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate on validation set
validation_accuracy = best_model.score(X_valid, y_valid)
print("Validation Accuracy:", validation_accuracy)

# Generate classification report
y_pred_valid = best_model.predict(X_valid)
report_valid = classification_report(y_valid, y_pred_valid)
print("Validation Classification Report:")
print(report_valid)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Validation Accuracy: 0.7844036697247706
Validation Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.67      0.75       428
           1       0.74      0.89      0.81       444

    accuracy                           0.78       872
   macro avg       0.80      0.78      0.78       872
weighted avg       0.80      0.78      0.78       872



In [19]:
X_test = test_df['text_stemmed']
y_test = test_df['label']

# Evaluate on test set
test_accuracy = best_model.score(X_test, y_test)
print("\nTest Accuracy:", test_accuracy)

# Generate classification report for test set
y_pred_test = best_model.predict(X_test)
report_test = classification_report(y_test, y_pred_test)
print("Test Classification Report:")
print(report_test)

# Print best hyperparameters
print("\nBest Hyperparameters:", best_params)


Test Accuracy: 0.78156
Test Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.64      0.75     25000
           1       0.72      0.92      0.81     25000

    accuracy                           0.78     50000
   macro avg       0.81      0.78      0.78     50000
weighted avg       0.81      0.78      0.78     50000


Best Hyperparameters: {'logisticregression__C': 10.0, 'logisticregression__class_weight': None, 'logisticregression__penalty': 'l2', 'logisticregression__solver': 'liblinear', 'tfidfvectorizer__ngram_range': (1, 3)}
